In [1]:
#!unzip Train.zip
#!unzip Test.zip

# This cell just needs to be run once. 
# After running it, you can comment it by adding # as the first character in the two first lines.


# # Importing the neccessary libraries 

In [ ]:
import sys
from matplotlib import pyplot
import numpy as np

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_seed
from tensorflow.keras.activations import sigmoid
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
import itertools

#This command makes only 1 GPU visible to our code, in case we have multiple GPUs.
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# To have reproducible results, it is important to set the seed for all tensorflow functions
set_seed(1020)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/lesliecook/opt/anaconda3/envs/tensorflow1X/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/k3/241jcrfd3_v3083mjlls6xsm0000gn/T/ipykernel_3108/3246960550.py", line 10, in <module>
    from tensorflow.random import set_seed
ImportError: cannot import name 'set_seed' from 'tensorflow.random' (/Users/lesliecook/opt/anaconda3/envs/tensorflow1X/lib/python3.7/site-packages/tensorflow_core/_api/v1/random/__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lesliecook/opt/anaconda3/envs/tensorflow1X/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ImportError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception

## defining variables

In [7]:
# Configure variables

# Here we define the folders where our dataset is saved.
TRAIN_DATA_DIR = r'Train/Train'
TEST_DATA_DIR = r'Test/Test'

# Here we define the input size of our neural network.
# Images will be resized automatically.
IMG_HEIGHT = 331
IMG_WIDTH = 331

# Here we define the size of the Batch to leverage
# the capacity GPUs have to parallelize 
BATCH_SIZE = 64

## funciton to plot learning curves

In [8]:
# This function plots the learning curves, which includes
# the loss curve and the accuracy curve over the epochs.
def plot_learning_curves(history):
    # plot loss
    pyplot.figure(figsize=(10, 5))
    pyplot.title('Binary Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='Train')
    pyplot.plot(history.history['val_loss'], color='orange', label='Validation')
    pyplot.legend(loc='upper right', shadow=True, fontsize='x-large')
    pyplot.show()
    # plot accuracy
    pyplot.figure(figsize=(10, 5))
    pyplot.title('Binary Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='Train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='Validation')
    pyplot.legend(loc='lower right', shadow=True, fontsize='x-large')
    pyplot.show()

## create CNN (convolutional neural network)

In [9]:
# Every model needs to have an input
x1 = Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))

# Below, we add paired convolutional and maxpooling layers
x = Conv2D(16, (3,3), activation='relu')(x1)
x = MaxPooling2D()(x)
x = Conv2D(16, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(32, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(32, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)

# Then we flatten the last vector
flat1 = Flatten()(x)
# Insert a dropout layer with 20% probability
flat2 = Dropout(0.2)(flat1)
# Then a dense layer
class1 = Dense(64, activation='relu', kernel_initializer='he_uniform')(flat2)
# And the output layer
class1b = Dense(1, activation='linear')(class1)
# The output needs to be binary, so we apply a sigmoid function
output = sigmoid(class1b)

# Here is where the model is created based on the input and output define above
model = Model(inputs=x1, outputs=output)

# We choose an optimizer
opt = Adam(lr=0.0005)

# The last step is to compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# We can see the structure and number of parameter of our network
# by calling .summary()
model.summary()

NameError: name 'Input' is not defined

## Data Generator: function to read images from folders to use for training and validation

In [10]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   #shear_range=0.2,
                                   #zoom_range=0.2,
                                   #horizontal_flip=False,
                                   #vertical_flip=True,
                                   #rotation_range=0,
                                   #fill_mode='constant',
                                   #cval=0,
                                   #preprocessing_function=preprocess_input,
                                   validation_split=0.2)  # set validation split

train_it = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    subset='training')  # set as training data

val_it = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,  # same directory as training data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    subset='validation')  # set as validation data

NameError: name 'ImageDataGenerator' is not defined

## Training the network

In [11]:
# The .fit() method is used to train our network
# You can specify here the number os epochs
history = model.fit(train_it, steps_per_epoch=len(train_it), 
                              validation_data=val_it, validation_steps=len(val_it), 
                              epochs=15, verbose=1)

plot_learning_curves(history)

NameError: name 'model' is not defined

## save the model in a file to be used later

In [12]:
# To save your model, uncomment the following line and run this cell.

#model.save('SimpleGadClass.h5')

## reading the trained model

In [13]:
# Make sure the file name you try to read from is the same you saved

model = load_model('SimpleGadClass.h5')

NameError: name 'load_model' is not defined

## validation set predictions for performance metrics calculations

In [14]:
i=0
y_true = []
y_pred = []
x_test = []

for x, y in val_it:
    y_true.extend(y)
    y_pred.extend(model.predict(x))
    x_test.extend(x)
    i+=1
    if i==len(val_it):
        break

y_pred = np.asarray(y_pred)
x_test = np.asarray(x_test)

NameError: name 'val_it' is not defined

## plot  ROC curve for validation set

In [15]:
 fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_pred)

auc_keras = auc(fpr_keras, tpr_keras)

pyplot.figure(figsize=(10,7))
pyplot.plot([0, 1], [0, 1], 'k--')
pyplot.plot(fpr_keras, tpr_keras, label='AUC (area = {:.5f})'.format(auc_keras), color='orange')
pyplot.xlabel('False positive rate')
pyplot.ylabel('True positive rate')
pyplot.title('ROC curve')
pyplot.legend(loc='best')
pyplot.show()

NameError: name 'roc_curve' is not defined

## plot confusion matrix for validation set

In [16]:
thresh = 0.5
cm = confusion_matrix(y_true, y_pred > thresh)

pyplot.figure(figsize=(7,7))
pyplot.imshow(cm, cmap=pyplot.cm.Blues)
pyplot.xlabel("Predicted labels")
pyplot.ylabel("True labels")
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    pyplot.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > 120 else "black", size='x-large')
pyplot.xticks([], [])
pyplot.yticks([], [])
pyplot.title('Confusion matrix ')
pyplot.colorbar()
pyplot.show()

NameError: name 'confusion_matrix' is not defined

## test set predictions for performance metrics calculations

In [17]:
ext_val_datagen = ImageDataGenerator(rescale=1. / 255,
                                   #shear_range=0.2,
                                   #zoom_range=0.2,
                                   #horizontal_flip=False,
                                   #vertical_flip=True,
                                   #rotation_range=0,
                                   #fill_mode='constant',
                                   #cval=0,
                                   #preprocessing_function=preprocess_input,
                                   )  # set validation split

ext_val_it = ext_val_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb')  # set as training data

NameError: name 'ImageDataGenerator' is not defined

In [18]:
i=0
y_true = []
y_pred = []
x_test = []

for x, y in ext_val_it:

    y_true.extend(y)
    y_pred.extend(model.predict(x))
    x_test.extend(x)
    i+=1
    if i==len(ext_val_it):
        break

y_pred = np.asarray(y_pred)
x_test = np.asarray(x_test)

NameError: name 'ext_val_it' is not defined

## plot ROC curve for the test set

In [19]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_pred)

auc_keras = auc(fpr_keras, tpr_keras)

pyplot.figure(figsize=(10,7))
pyplot.plot([0, 1], [0, 1], 'k--')
pyplot.plot(fpr_keras, tpr_keras, label='AUC (area = {:.5f})'.format(auc_keras), color='orange')
pyplot.xlabel('False positive rate')
pyplot.ylabel('True positive rate')
pyplot.title('ROC curve')
pyplot.legend(loc='best')
pyplot.show()

NameError: name 'roc_curve' is not defined

## best accuaracy threshold

In [20]:
thresh = thr_list[acc_list.index(max(acc_list))]
cm = confusion_matrix(y_true, y_pred > thresh)

pyplot.figure(figsize=(7,7))
pyplot.imshow(cm, cmap=pyplot.cm.Blues)
pyplot.xlabel("Predicted labels")
pyplot.ylabel("True labels")
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    pyplot.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > 120 else "black", size='x-large')
pyplot.xticks([], [])
pyplot.yticks([], [])
pyplot.title('Confusion matrix ')
pyplot.colorbar()
pyplot.show()

NameError: name 'thr_list' is not defined

## plot incorrectly predicted models to understand errors and find solutions

In [21]:
for i in range(len(y_true)):
    if y_true[i] != 1. * (y_pred[i, 0] > thresh):
        print('Truth:' + str(y_true[i]))
        print('Pred:' + str(y_pred[i, 0]))
        print(i)
        pyplot.figure(figsize=(7,7))
        pyplot.imshow(x_test[i])
        pyplot.show()